In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from datetime import datetime, timedelta
import pandas as pd 


current_date = pd.to_datetime(datetime.utcnow()).floor("H") - timedelta(28*7)
print(f"{current_date=}")

current_date=Timestamp('2022-11-06 04:00:00')


In [7]:
from src.inference import loadFeatBatchFromStore

features = loadFeatBatchFromStore(current_date)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/42132
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2022-02-27 04:00:00 to 2022-03-27 04:00:00


2023-05-20 21:27:04,446 INFO: USE `ml_prod_pipline_featurestore`
2023-05-20 21:27:05,118 INFO: SELECT `fg0`.`pickup_hr` `pickup_hr`, `fg0`.`rides` `rides`, `fg0`.`pickup_loc_id` `pickup_loc_id`
FROM `ml_prod_pipline_featurestore`.`ts_hourly_feat_group_1` `fg0`
WHERE `fg0`.`pickup_hr` >= TIMESTAMP '2022-02-26 04:00:00.000' AND `fg0`.`pickup_hr` <= TIMESTAMP '2022-03-28 04:00:00.000'


Index(['pickup_hr', 'rides', 'pickup_loc_id'], dtype='object')
262
176326 176326
ts_data=                 pickup_hr  rides  pickup_loc_id
62289  2022-02-27 04:00:00      0              1
11136  2022-02-27 05:00:00      0              1
97827  2022-02-27 06:00:00      0              1
121408 2022-02-27 07:00:00      0              1
20207  2022-02-27 08:00:00      0              1
...                    ...    ...            ...
167878 2022-03-27 00:00:00      6            265
138125 2022-03-27 01:00:00      5            265
43176  2022-03-27 02:00:00      4            265
52625  2022-03-27 03:00:00      5            265
68     2022-03-27 04:00:00      1            265

[176326 rows x 3 columns]


In [8]:
from src.inference import (
    loadModelFromRegistry,
    getModelPredictions
)

model = loadModelFromRegistry()
predictions = getModelPredictions(model, features)

predictions['pickup_hr'] = current_date

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/42132
Connected. Call `.close()` to terminate connection gracefully.
after adding avg rides (262, 675)


In [9]:
predictions

,pickup_loc_id,predicted_demand,pickup_hr
0,121,-0.0,2022-11-06 04:00:00
1,148,10.0,2022-11-06 04:00:00
2,79,47.0,2022-11-06 04:00:00
3,187,-0.0,2022-11-06 04:00:00
4,107,48.0,2022-11-06 04:00:00
...,...,...,...
257,163,42.0,2022-11-06 04:00:00
258,169,-0.0,2022-11-06 04:00:00
259,238,32.0,2022-11-06 04:00:00
260,183,-0.0,2022-11-06 04:00:00


In [10]:
from src.feature_store_api import getFeatureStore
import src.config as config

# Connect to feature group

feature_group = getFeatureStore().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Model Generated Predictions",
    primary_key=["pickup_loc_id", "pickup_hr"],
    event_time="pickup_hr"
)

feature_group.insert(predictions, write_options={"wait_for_job": False})

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/42132
Connected. Call `.close()` to terminate connection gracefully.


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/42132/fs/42052/fg/50528


Uploading Dataframe: 0.00% |          | Rows 0/262 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/42132/jobs/named/model_predictions_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f9d472b9b20>, None)